In [3]:
import pandas as pd, numpy as np
import os, io, datetime, re, json
from hdfs import InsecureClient
import getpass
import fsspec


In [4]:
fs = fsspec.filesystem(
    'webhdfs',
    host='localhost',
    port=9870,
    user='user'
)


In [5]:
file_name = 'HPG_Capital_Increase_M&A.json'

In [9]:
file_path = f'/datalake/raw/{file_name}'
with fs.open(file_path, 'r', encoding='utf-8') as f:
    js = json.load(f)

In [11]:
def convert_ms_date(ms_date_str):
    match = re.search(r'\d+', ms_date_str)  # Tìm số trong chuỗi /Date(...)/
    if match:
        timestamp_ms = int(match.group())   # Lấy số, chuyển thành int (milliseconds)
        timestamp_s = timestamp_ms / 1000   # Chuyển sang seconds
        dt = datetime.datetime.utcfromtimestamp(timestamp_s)    # Tạo datetime từ Unix timestamp
        return dt.strftime('%Y-%m-%d %H:%M:%S') # Định dạng lại thành chuỗi dễ đọc
    return ms_date_str  # Trường hợp không đúng định dạn

In [14]:
def json_to_parquet_publishtime(input_path, output_path):
    """
       input_path (str): Đường dẫn file JSON đầu vào.
       output_path (str): Đường dẫn file Parquet đầu ra.
    """
    #chuyển định dạng thời gian


    #mở file json
    with fs.open(input_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    #lấy dữ liệu
    filtered_data = []
    for item in data:
        filtered_item = {
            "Title": item.get("Title"),
            "PublishTime": convert_ms_date(item.get("PublishTime", ""))
        }
        filtered_data.append(filtered_item)

    df = pd.DataFrame(filtered_data)

    #tạo file parquet
    with fs.open(output_path, 'wb') as f:
        df.to_parquet(f, engine='pyarrow', index=False)

    return df


In [16]:
file_name = ['HPG_Capital_Increase_M&A', 'HPG_Devidend', 'HPG_industry_news', 'HPG_Personnel_changes', 'HPG_Shareholder_transactions', 'HPG_Results_Business_Plan']
for file in file_name:
    json_to_parquet_publishtime(
        input_path = f'/datalake/raw/{file}.json',
        output_path = f'/datalake/cleaned/{file}.parquet'
    )

In [18]:
file_name = ['HSG_Capital_Increase_M&A', 'HSG_Devidend', 'HSG_industry_news', 'HSG_Personnel_changes', 'HSG_Shareholder_transactions', 'HSG_Results_Business_Plan']
for file in file_name:
    json_to_parquet_publishtime(
        input_path = f'/datalake/raw/{file}.json',
        output_path = f'/datalake/cleaned/{file}.parquet'
    )